In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os

BASE_PATH = "/content/drive/MyDrive/risklens/"
DATA_PATH = os.path.join(BASE_PATH, "dataset")

app = pd.read_csv(os.path.join(DATA_PATH, "application_train.csv"))

app.shape


(307511, 122)

In [4]:
core_cols = [
    "SK_ID_CURR",
    "TARGET",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "CNT_FAM_MEMBERS"
]

df = app[core_cols].copy()
df.head()


,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
0,100002,1,202500.0,406597.5,24700.5,-9461,-637,1.0
1,100003,0,270000.0,1293502.5,35698.5,-16765,-1188,2.0
2,100004,0,67500.0,135000.0,6750.0,-19046,-225,1.0
3,100006,0,135000.0,312682.5,29686.5,-19005,-3039,2.0
4,100007,0,121500.0,513000.0,21865.5,-19932,-3038,1.0


In [5]:
df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)

/tmp/ipython-input-2902537088.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)


In [6]:
df["age_years"] = -df["DAYS_BIRTH"] / 365

df["loan_to_income"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]

df["annuity_to_income"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]

df["income_per_person"] = df["AMT_INCOME_TOTAL"] / df["CNT_FAM_MEMBERS"]

df["employment_ratio"] = df["DAYS_EMPLOYED"] / df["DAYS_BIRTH"]


In [7]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
risk_check = df.groupby("TARGET")[
    ["loan_to_income", "annuity_to_income", "employment_ratio"]
].mean()

risk_check


,loan_to_income,annuity_to_income,employment_ratio
TARGET,,,
0,3.963729,0.180530,0.159656
1,3.887438,0.185482,0.127388


In [9]:
df["loan_to_income_bucket"] = pd.qcut(
    df["loan_to_income"],
    q=5,
    duplicates="drop"
)


In [10]:
df.groupby("loan_to_income_bucket")["TARGET"].mean()


/tmp/ipython-input-584701189.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("loan_to_income_bucket")["TARGET"].mean()


,TARGET
loan_to_income_bucket,
"(0.00381, 1.818]",0.073352
"(1.818, 2.764]",0.085672
"(2.764, 3.906]",0.089253
"(3.906, 5.769]",0.082839
"(5.769, 84.737]",0.072538


In [12]:
FINAL_COLS = [
    "SK_ID_CURR",
    "TARGET",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "CNT_FAM_MEMBERS",
    "age_years",
    "loan_to_income",
    "annuity_to_income",
    "income_per_person",
    "employment_ratio",
    "loan_to_income_bucket"
]

df_day2 = df[FINAL_COLS].copy()

OUTPUT_PATH = "/content/drive/MyDrive/risklens/data/processed"
os.makedirs(OUTPUT_PATH, exist_ok=True)

df_day2.to_csv(
    f"{OUTPUT_PATH}/application_features_day2.csv",
    index=False
)


In [13]:
pd.read_csv(
    f"{OUTPUT_PATH}/application_features_day2.csv"
).shape


(307511, 12)

Loan-to-income ratio shows non-monotonic default behavior; therefore, both raw and bucketed versions are retained to capture non-linear risk patterns.

# ***Bureau & Credit History Features***

In [14]:
import pandas as pd
import numpy as np
import os

BASE_PATH = "/content/drive/MyDrive/risklens/"
DATASET_PATH = os.path.join(BASE_PATH, "dataset")
PROCESSED_PATH = os.path.join(BASE_PATH, "data/processed")

app_day2 = pd.read_csv(
    os.path.join(PROCESSED_PATH, "application_features_day2.csv")
)

app_day2.shape


(307511, 12)

In [15]:
bureau = pd.read_csv(
    os.path.join(DATASET_PATH, "bureau.csv")
)

bureau_balance = pd.read_csv(
    os.path.join(DATASET_PATH, "bureau_balance.csv")
)

bureau.shape, bureau_balance.shape


((1716428, 17), (27299925, 3))

In [21]:
status_map = {
    "C": 0,
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "X": 0
}

bureau_balance["STATUS_NUM"] = bureau_balance["STATUS"].map(status_map)

bb_agg = bureau_balance.groupby("SK_ID_BUREAU").agg(
    max_delinquency=("STATUS_NUM", "max"),
    months_with_delinquency=("STATUS_NUM", lambda x: (x > 0).sum())
).reset_index()


In [27]:
bb_agg.head()

,SK_ID_BUREAU,max_delinquency,months_with_delinquency
0,5001709,0,0
1,5001710,0,0
2,5001711,0,0
3,5001712,0,0
4,5001713,0,0


In [22]:
bureau = bureau.merge(
    bb_agg,
    on="SK_ID_BUREAU",
    how="left"
)


In [29]:
bureau_client_agg = bureau.groupby("SK_ID_CURR").agg(
    num_bureau_loans=("SK_ID_BUREAU", "count"),
    num_active_loans=("CREDIT_ACTIVE", lambda x: (x == "Active").sum()),
    num_closed_loans=("CREDIT_ACTIVE", lambda x: (x == "Closed").sum()),
    total_credit_exposure=("AMT_CREDIT_SUM", "sum"),
    total_credit_debt=("AMT_CREDIT_SUM_DEBT", "sum"),
    max_delinquency=("max_delinquency", "max"),
    total_delinquency_months=("months_with_delinquency", "sum")
).reset_index()

bureau_client_agg.head()



,SK_ID_CURR,num_bureau_loans,num_active_loans,num_closed_loans,total_credit_exposure,total_credit_debt,max_delinquency,total_delinquency_months
0,100001,7,3,4,1453365.000,596686.5,1.0,1.0
1,100002,8,2,6,865055.565,245781.0,1.0,27.0
2,100003,4,1,3,1017400.500,0.0,NaN,0.0
3,100004,2,0,2,189037.800,0.0,NaN,0.0
4,100005,3,2,1,657126.000,568408.5,0.0,0.0


In [32]:
missing_pct = bureau_client_agg["num_bureau_loans"].isna().mean() * 100
print(f"Missing num_bureau_loans: {missing_pct:.2f}%")

bureau_client_agg.shape

Missing num_bureau_loans: 0.00%


(305811, 8)

In [24]:
app_day4 = app_day2.merge(
    bureau_client_agg,
    on="SK_ID_CURR",
    how="left"
)


In [25]:
app_day4[
    ["num_bureau_loans", "max_delinquency", "total_delinquency_months"]
].isna().mean()


,0
num_bureau_loans,0.143149
max_delinquency,0.700073
total_delinquency_months,0.143149


In [26]:
app_day4.to_csv(
    os.path.join(PROCESSED_PATH, "application_bureau_features_day4.csv"),
    index=False
)


# Insights
1. Shape (bureau.csv (1716428, 17), bureau_balance.csv(27299925, 3))
2. bureau_client_agg.shape (305811, 8)
3.